In [16]:
import matplotlib, time, copy
import matplotlib.pyplot as plt
import autograd.numpy as np
import autograd.scipy.stats as sps_autograd
from autograd import grad, hessian
from statsmodels.tsa.arima_process import ArmaProcess
from scipy.optimize import minimize
from scipy.linalg import toeplitz

# Simulate ARMA data 

In [17]:
"""
Simulate ARMA(1, 1) model
"""

# Define AR and MA coefficients
ar = np.array([1, -0.5])  
ma = np.array([1, 0.4])        

# Create ARMA process object
arma_process = ArmaProcess(ar, ma)

# Simulate 10000 samples
N = 1000
y = arma_process.generate_sample(nsample=N)

# Karman Filter 

In [18]:
def initialize_FGHQ(a, b):
    """
    Construct the state-space matrices F, G, H for an ARMA(p, q) model.

    Parameters:
    - p: int, order of the AR component
    - q: int, order of the MA component
    - a: list or np.array of AR coefficients [a1, a2, ..., ap]
    - b: list or np.array of MA coefficients [b1, b2, ..., bq]

    Returns:
    - F: state transition matrix of shape (k, k)
    - G: noise coefficient matrix of shape (k, 1)
    - H: observation matrix of shape (1, k)
    - Q: covariance identity matrix of shape (k, k)
    """
    p = len(a)
    q = len(b)
    k = max(p, q + 1)  # dimension of the state vector
    F = np.zeros((k, k))
    G = np.zeros((k, 1))
    H = np.zeros((1, k))

    # Fill the first column of F with AR coefficients
    for i in range(p):
        F[i, 0] = a[i]

    # Fill the lower subdiagonal of F with 1s (shifting the state)
    for i in range(k - 1):
        F[i, i + 1] = 1

    # Fill G with negative MA coefficients, first element = 1
    for i in range(q):
        G[i+1, 0] = -b[i]
    G[0, 0] = 1  # first element always set to 1

    # Matrix H: only first element is 1
    H[0, 0] = 1

    # Initialize covariance matrix Q as identity matrix
    Q = np.eye(k)

    return F, G, H, Q

In [19]:
def log_likelihood(sigma2_hat, r):
    N = len(r)
    return -0.5 * (N * np.log(2 * np.pi) 
                   + N * np.log(sigma2_hat) 
                   + np.sum(np.log(r)) + N)

In [20]:
def karman_filter_arma(theta):
    p = 1   # AR order
    q = 1   # MA order
    a = theta[:p]
    b = theta[p:p+q]
    k = max(p, q + 1)
    
    F, G, H, Q = initialize_FGHQ(a, b)

    # Initialize values
    x = np.zeros((k, 1))
    V = np.eye(k) * 100
    e = np.zeros((N, 1))
    r = np.zeros((N, 1))

    # Implement Kalman filter
    for t in range(N):
        # Predict one-step-ahead state predictive density of x_{t}
        x_predict = F @ x
        V_predict = F @ V @ F.T + G @ G.T

        # Compute forecast error and one-step-ahead predictive variance
        e[t] = y[t] - (H @ x_predict).item()
        r[t] = (H @ V_predict @ H.T).item()

        # Kalman gain
        K = V_predict @ H.T / r[t]

        # Update current state and covariance
        x = x_predict + K * e[t]
        V = (np.eye(k) - K @ H) @ V_predict

    sigma2_hat = np.sum(e**2 / r) / N

    return sigma2_hat, r

In [21]:
def obj_func_likelihood(theta):
    sigma2_hat, r = karman_filter_arma(theta)
    log_lik = log_likelihood(sigma2_hat, r)

    return -log_lik  

In [22]:
"""
Test the log-likelihood function
"""

theta_start = [0.1, 0.1]

# Minimize negative log-likelihood
result = minimize(obj_func_likelihood, theta_start, method='BFGS')

# Print results
print("Estimated parameters:", result.x)
print("Negative log-likelihood:", result.fun)

Estimated parameters: [ 0.49743377 -0.39634635]
Negative log-likelihood: 141605.20910991152


# Test algorithm

In [ ]:
def initialize_FGHQ(a, b):
    """
    Construct the state-space matrices F, G, H for an ARMA(p, q) model.

    Parameters:
    - p: int, order of the AR component
    - q: int, order of the MA component
    - a: list or np.array of AR coefficients [a1, a2, ..., ap]
    - b: list or np.array of MA coefficients [b1, b2, ..., bq]
    - l: int, total number of parameters

    Returns:
    - F: state transition matrix of shape (k, k)
    - G: noise coefficient matrix of shape (k, 1)
    - H: observation matrix of shape (1, k)
    - Q: covariance identity matrix of shape (k, k)
    - dF_dtheta: derivative of F with respect to theta of shape (l, k, k)
    - dG_dtheta: derivative of G with respect to theta of shape (l, k, 1)
    """
    p = len(a)
    q = len(b)
    k = max(p, q + 1)  # dimension of the state vector
    F = np.zeros((k, k))
    G = np.zeros((k, 1))
    H = np.zeros((1, k))

    # Fill the first column of F with AR coefficients
    for i in range(p):
        F[i, 0] = a[i]

    # Fill the lower subdiagonal of F with 1s (shifting the state)
    for i in range(k - 1):
        F[i, i + 1] = 1

    # Fill G with negative MA coefficients, first element = 1
    for i in range(q):
        G[i+1, 0] = -b[i]
    G[0, 0] = 1  # first element always set to 1

    # Matrix H: only first element is 1
    H[0, 0] = 1

    # Initialize covariance matrix Q as identity matrix
    Q = np.eye(k)

    # Compute derivatives of F and G with respect to theta
    dF_dtheta = np.zeros((k, k, k))
    dF_dtheta[0, 0, 0] = 1 # ARMA(1,1)

    dG_dtheta = np.zeros((k, k, 1))
    dG_dtheta[1, 1, 0] = -1 # ARMA(1,1)

    dGT_dtheta = np.zeros((k, 1, k))
    dGT_dtheta[1, 0, 1] = -1 # ARMA(1,1)

    return F, G, H, Q, dF_dtheta, dG_dtheta, dGT_dtheta 

In [ ]:
def compute_initial_V_and_dV(a, b, sigma2):
    """
    Tính toán V ban đầu và dV/d(theta) ban đầu.
    Thực thi các phương trình từ mục 3.3.2 của tài liệu.
    """
    p, q = 1, 1
    k = max(p, q + 1)  

    # 1. Tính hàm đáp ứng xung g_k (Eq. 52)
    g = np.zeros(k)
    g[0] = 1.0
    g[1] = a - b
    g[2] = a * g[1]

    C = np.zeros(k + q)
    C[0] = sigma2 * (1 - 2 * a * b + b**2) / (1 - a**2)
    C[1] = a * C[0] - sigma2 * b
    C[2] = a * C[1]

    V = np.zeros((k, k))
    V[0, 0] = C[0]
    V[0, 1] = V[1, 0] = - b * g[0]
    V[1, 1] = b**2 * sigma2

    dV_dtheta = np.zeros((k, k, k))
    dV_dtheta[0, 0, 0] = (2 * sigma2 * (a-b) * (1 - a*b)) / (1 - a**2)**2
    dV_dtheta[0, 0, 1] = 0
    dV_dtheta[0, 1, 0] = 0
    dV_dtheta[0, 1, 1] = 0
    dV_dtheta[1, 0, 0] = 2 * sigma2 * (b-a) / (1 - a**2)
    dV_dtheta[1, 0, 1] = -1
    dV_dtheta[1, 1, 0] = -1
    dV_dtheta[1, 1, 1] = 2 * b * sigma2

    return V, dV_dtheta

In [ ]:
def grad_obj_func_likelihood(theta):
    """
    Implement the gradient of the log-likelihood function for ARMA(1,1) model.
    """
    
    a = theta[0]
    b = theta[1]
    k = 2

    # Initialize the state-space matrices
    F, G, H, Q, dF_dtheta, dG_dtheta, dGT_dtheta = initialize_FGHQ([a], [b])
    
    # Initialize the x and V matrices
    V, dV_dtheta = compute_initial_V_and_dV(a, b, 1)
    x = np.zeros((k, 1))
    dx_n_n = np.zeros((2, 2, 1))
    e = np.zeros((N, 1))
    r = np.zeros((N, 1))

    # Các biến lưu trữ
    sum_term1 = np.zeros(p)
    sum_term2 = np.zeros(p)
    sum_term3 = np.zeros(p)
    sigma2_hat_num = 0

    # 
    for t in range(N):
        # 1. Kalman filter
        # Predict one-step-ahead state predictive density of x_{t}
        x_predict = F @ x
        V_predict = F @ V @ F.T + G @ G.T

        # Compute forecast error and one-step-ahead predictive variance
        e[t] = y[t] - (H @ x_predict).item()
        r[t] = (H @ V_predict @ H.T).item()

        # Kalman gain
        K = V_predict @ H.T / r[t]

        # Update current state and covariance
        x = x_predict + K * e[t]
        V = (np.eye(k) - K @ H) @ V_predict

        sigma2_hat_num += (epsilon_n**2 * r_n_inv)[0, 0]

        # 2. Kalman filter for gradient
        dx_dtheta = F @ dx_n_n + dF_dtheta @ x

        dF_dtheta_T = dF_dtheta.transpose(0, 2, 1)
        dG_dtheta_T = dG_dtheta.transpose(0, 2, 1)
        
        term1_dV = F @ dV_n_n @ F.T
        term2_dV = dF_dtheta @ V_n_n @ F.T
        term3_dV = (F @ V_n_n) @ dF_dtheta_T
        term4_dV = (dG_dtheta * sigma2) @ G.T
        term5_dV = (G * sigma2) @ dG_dtheta_T
        
        dV_n_n_1 = term1_dV + term2_dV + term3_dV + term4_dV + term5_dV

        d_epsilon_n = -H @ dx_n_n_1
        d_r_n = H @ dV_n_n_1 @ H.T

        r_n_inv_2 = r_n_inv**2
        d_K_n = (dV_n_n_1 @ H.T * r_n_inv) - (V_n_n_1 @ H.T * r_n_inv_2 * d_r_n)

        dx_n_n = dx_n_n_1 + K_n @ d_epsilon_n + d_K_n @ epsilon_n
        
        dV_n_n = dV_n_n_1 - (d_K_n @ H @ V_n_n_1) - (K_n @ H @ dV_n_n_1)
        
        # Cập nhật tổng
        sum_term1 += (r_n_inv * d_r_n).reshape(p)
        sum_term2 += (epsilon_n[0,0] * r_n_inv * d_epsilon_n).reshape(p)
        sum_term3 += ((epsilon_n[0,0]**2 * r_n_inv_2) * d_r_n).reshape(p)

    # === 3. Tính Gradient cuối cùng ===
    sigma2_hat = np.sum(e**2 / r) / N
    
    grad = -0.5 * sum_term1 - (1/sigma2_hat) * sum_term2 + (1/(2*sigma2_hat)) * sum_term3
    
    return grad

In [ ]:
"""
Test the log-likelihood function
"""

theta_start = [0.1, 0.1]

# Minimize negative log-likelihood
result = minimize(obj_func_likelihood, theta_start, method='BFGS', jac=grad_obj_func_likelihood,
                  options={'gtol': 1e-04, 'maxiter': 1000, 'disp': True})

# Print results
print("Estimated parameters:", result.x)
print("Negative log-likelihood:", result.fun)

array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])

In [40]:
np.zeros(2)

array([0., 0.])

In [41]:
len(1)

TypeError: object of type 'int' has no len()